#### Load images from test and train directories, re-size just in case there are odd-sized images.

In [1]:
import graphlab as gl
WORKING_DIR = '/home/wilber/work/Galvanize/gcp-data/lunarPhotos/100x100'
train_sf = gl.image_analysis.load_images(WORKING_DIR + '/train',
                                         random_order=True)
train_sf['image'] = gl.image_analysis.resize(train_sf['image'], 256, 256)
test_sf = gl.image_analysis.load_images(WORKING_DIR + '/test',
                                        random_order=True)
test_sf['image'] = gl.image_analysis.resize(test_sf['image'], 256, 256)

[INFO] This trial license of GraphLab Create is assigned to trephine1@yahoo.com and will expire on October 12, 2015. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-20561 - Server binary: /home/wilber/work/Galvanize/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1442272918.log
[INFO] GraphLab Server Version: 1.5.2


#### Generate labels

In [2]:
train_sf['label'] = train_sf['path'].apply(lambda x: 1 if x.split('/')[-1].startswith('h') else 0)
test_sf['label'] = test_sf['path'].apply(lambda x: 1 if x.split('/')[-1].startswith('h') else 0)
print train_sf['label'].head(40)
print test_sf['label'].head(40)
train_sf.save(WORKING_DIR + '/sframe/train_shuffle')
test_sf.save(WORKING_DIR + '/sframe/test_shuffle')

[0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0]
[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1]


In [3]:
gl.canvas.set_target('ipynb')
train = gl.SFrame(WORKING_DIR + '/sframe/train_shuffle')
train.head()

[WARNING] This Python session does not appear to be running in an interactive IPython Notebook. Use of the 'ipynb' target may behave unexpectedly or result in errors.


path,image,label
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,0
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,1
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,1
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,1
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,0
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,0
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,1
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,1
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,1
/home/wilber/work/Galvanize/gcp-data/lunarPhot ...,Height: 256 Width: 256,1


#### subtract mean image from training data:

In [4]:
# mean_image = train['image'].mean()
# gl.SArray([mean_image]).save(WORKING_DIR + '/sframe/mean_image')
mean_image = gl.SArray(WORKING_DIR + '/sframe/mean_image')[0]
mean_image

Height: 256px
Width: 256px
Channels: 3

#### Obtain a NerualNet object from the builtin networks in the deeplearing toolkit.
#### The "imagenet" NeuralNet is derived from Krizhevsky, Alex, Ilya Sutskever, and Geoffrey E. Hinton, 2012.

In [5]:
net = gl.deeplearning.get_builtin_neuralnet('imagenet')
net


### network layers ###
layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 4
  num_channels = 96
  num_groups = 1
  kernel_size = 11
layer[1]: RectifiedLinearLayer
layer[2]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[3]: LocalResponseNormalizationLayer
  alpha = 0.001
  beta = 0.75
  knorm = 1
  local_size = 5
layer[4]: ConvolutionLayer
  init_random = gaussian
  padding = 2
  stride = 1
  num_channels = 256
  num_groups = 2
  kernel_size = 5
layer[5]: RectifiedLinearLayer
layer[6]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[7]: LocalResponseNormalizationLayer
  alpha = 0.001
  beta = 0.75
  knorm = 1
  local_size = 5
layer[8]: ConvolutionLayer
  init_random = gaussian
  padding = 1
  stride = 1
  num_channels = 384
  num_groups = 1
  kernel_size = 3
layer[9]: RectifiedLinearLayer
layer[10]: ConvolutionLayer
  init_random = gaussian
  padding = 1
  stride = 1
  num_channels = 384
  num_groups = 2
  kernel_size = 

In [6]:
net.layers[-2].num_hidden_units = 2.
net.verify()

True

#### Train!

In [ ]:
import time
start = time.time()
m = gl.neuralnet_classifier.create(train[['image', 'label']],                      
                                   target='label',
#                                  input_shape=[100, 100, 3],
#                                  output_shape=2,
                                   network=net,                                    
                                   mean_image=mean_image,                          
                                   metric=['accuracy','precision'],                 
                                   max_iterations=35,                              
                                   model_checkpoint_path=WORKING_DIR
                                     + '/result/model_checkpoint',
                                   model_checkpoint_interval=5,                    
                                   batch_size=50)
totsecs = time.time() - start
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

In [ ]:
gl.canvas.set_target('ipynb')
test = gl.SFrame(WORKING_DIR + '/sframe/train_shuffle')
test.head()

In [ ]:
# Random split the training-data
training_data, validation_data = data.random_split(0.8)

# Make sure all images are of the same size (Required by Neuralnets)
for sf in [training_data, validation_data, test_data]:
  sf['image'] = gl.image_analysis.resize(sf['image'], 100, 100, 1)

net = gl.deeplearning.get_builtin_neuralnet('mnist')

print "Layers of the network "
print "--------------------------------------------------------"
print net.layers

print "Parameters of the network "
print "--------------------------------------------------------"
print net.params

In [ ]:
model = gl.neuralnet_classifier.create(training_data, target='label',
                                         network = net,
                                         validation_set=validation_data,
                                         metric=['accuracy', 'recall@2'],
                                         max_iterations=3)

In [ ]:
predictions = model.classify(test_data)
print predictions

In [ ]:
pred_top2 = model.predict_topk(test_data, k=2)
print pred_top2

In [ ]:
result = model.evaluate(test_data)
print "Accuracy         : %s" % result['accuracy']
print "Confusion Matrix : \n%s" % result['confusion_matrix']

### Comment: effing impressive. Training/optimizing an Octave-based model to do barely a hair better took me weeks.